In [26]:
from __future__ import absolute_import
import pandas as pd
import os
import sys
import bleu
import pickle
import torch
import json
import random
import logging
import argparse
import numpy as np
from io import open
from itertools import cycle
import torch.nn as nn
from model import Seq2Seq
from tqdm import tqdm, trange
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaModel, RobertaTokenizer)
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer)}

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

class Example(object):
    """A single training/test example."""
    def __init__(self,
                 idx,
                 source,
                 target,
                 ):
        self.idx = idx
        self.source = source
        self.target = target

def read_examples(filename):
    """Read examples from filename."""
    examples=[]
    print(filename)
    with open(filename,encoding="utf-8") as f:
        for idx, line in enumerate(f):
            line=line.strip()
            js=json.loads(line)
            if 'idx' not in js:
                try:
                    js['idx']=idx
                except:
                    print(idx)
            code=' '.join(js['code_tokens']).replace('\n',' ')
            code=' '.join(code.strip().split())
            nl=' '.join(js['docstring_tokens']).replace('\n','')
            nl=' '.join(nl.strip().split())            
            examples.append(
                Example(
                        idx = idx,
                        source=code,
                        target = nl,
                        ) 
            )

    return examples


class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask       
        


def convert_examples_to_features(examples, tokenizer, args,stage=None):
    features = []
    for example_index, example in enumerate(examples):
        #source
        source_tokens = tokenizer.tokenize(example.source)[:args.max_source_length-2]
        source_tokens =[tokenizer.cls_token]+source_tokens+[tokenizer.sep_token]
        source_ids =  tokenizer.convert_tokens_to_ids(source_tokens) 
        source_mask = [1] * (len(source_tokens))
        padding_length = args.max_source_length - len(source_ids)
        source_ids+=[tokenizer.pad_token_id]*padding_length
        source_mask+=[0]*padding_length
 
        #target
        if stage=="test":
            target_tokens = tokenizer.tokenize("None")
        else:
            target_tokens = tokenizer.tokenize(example.target)[:args.max_target_length-2]
        target_tokens = [tokenizer.cls_token]+target_tokens+[tokenizer.sep_token]            
        target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        target_mask = [1] *len(target_ids)
        padding_length = args.max_target_length - len(target_ids)
        target_ids+=[tokenizer.pad_token_id]*padding_length
        target_mask+=[0]*padding_length   
   
        if example_index < 5:
            if stage=='train':
                logger.info("*** Example ***")
                logger.info("idx: {}".format(example.idx))

                logger.info("source_tokens: {}".format([x.replace('\u0120','_') for x in source_tokens]))
                logger.info("source_ids: {}".format(' '.join(map(str, source_ids))))
                logger.info("source_mask: {}".format(' '.join(map(str, source_mask))))
                
                logger.info("target_tokens: {}".format([x.replace('\u0120','_') for x in target_tokens]))
                logger.info("target_ids: {}".format(' '.join(map(str, target_ids))))
                logger.info("target_mask: {}".format(' '.join(map(str, target_mask))))
       
        features.append(
            InputFeatures(
                 example_index,
                 source_ids,
                 target_ids,
                 source_mask,
                 target_mask,
            )
        )
    return features



def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
         

# parser = argparse.ArgumentParser()
## Required parameters  
# parser.add_argument("--load_model_path", default=None, type=str, 
#                     help="Path to trained model: Should contain the .bin files" )    
## Other parameters

# print arguments
# args = parser.parse_args()
class Args:
    def __init__(self):
        self.model_type = ''
        self.model_name_or_path = ''
        self.test_filename = ''
        self.output_dir = ''
        self.max_source_length = ''
        self.max_target_length = ''
        self.beam_size = ''
        self.eval_batch_size = ''
        self.local_rank = -1
        self.no_cuda = False
        self.seed = 42
        self.config_name = ''
        self.tokenizer_name = ''
        self.do_lower_case = False
        
args = Args()
args.model_type = 'roberta'
args.model_name_or_path ='microsoft/codebert-base'
args.test_filename = '/home/jovyan/Source_Code_Veri/data/GCJ/MS_C2T/gcj2017_java.jsonl'
args.output_dir = '/home/jovyan/Source_Code_Veri/data/GCJ/MS_C2T'
args.max_source_length = 256
args.max_target_length = 128
args.beam_size = 10
args.eval_batch_size = 10
args.load_model_path = None

logger.info(args)

#     np.save(os.path.join(args.output_dir,args.test_filename.split('/')[-1].split('.')[0]), save_all)



02/25/2021 18:47:56 - INFO - __main__ -   <__main__.Args object at 0x7f4510651390>


In [33]:
# Setup CUDA, GPU & distributed training
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend='nccl')
    args.n_gpu = 1
logger.warning("Process rank: %s, device: %s, n_gpu: %s, distributed training: %s",
                args.local_rank, device, args.n_gpu, bool(args.local_rank != -1))
args.device = device
# Set seed
set_seed(args.seed)
# make dir if output_dir not exist
if os.path.exists(args.output_dir) is False:
    os.makedirs(args.output_dir)

config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,do_lower_case=args.do_lower_case)

#budild model
encoder = model_class.from_pretrained(args.model_name_or_path,config=config)    
decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model=Seq2Seq(encoder=encoder,decoder=decoder,config=config,
              beam_size=args.beam_size,max_length=args.max_target_length,
              sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)
if args.load_model_path is not None:
    logger.info("reload model from {}".format(args.load_model_path))
    model.load_state_dict(torch.load(args.load_model_path))

model.to(device)


files=[]

if args.test_filename is not None:
    files.append(args.test_filename)
print('===== @#@!@##### ======')
print(files)
print('===== @#@!@##### ======')
for idx,file in enumerate(files):   
    logger.info("Test file: {}".format(file))
    eval_examples = read_examples(file)
    eval_features = convert_examples_to_features(eval_examples, tokenizer, args,stage='test')
    all_source_ids = torch.tensor([f.source_ids for f in eval_features], dtype=torch.long)
    all_source_mask = torch.tensor([f.source_mask for f in eval_features], dtype=torch.long)    
    eval_data = TensorDataset(all_source_ids,all_source_mask)   

    # Calculate bleu
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)

    model.eval() 
    p=[]
    for batch in tqdm(eval_dataloader,total=len(eval_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        source_ids,source_mask= batch                  
        with torch.no_grad():
            preds = model(source_ids=source_ids,source_mask=source_mask)  
            for pred in preds:
                t=pred[0].cpu().numpy()
                t=list(t)
                if 0 in t:
                    t=t[:t.index(0)]
                text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
                p.append(text)
    model.train()
    predictions=[] 

02/25/2021 18:51:57 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 2, distributed training: False
02/25/2021 18:51:58 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/microsoft/codebert-base/config.json from cache at /home/jovyan/.cache/torch/transformers/1b62771d5f5169b34713b0af1ab85d80e11f7b1812fbf3ee7d03a866c5f58e72.06eb31f0a63f4e8a136733ccac422f0abf9ffa87c3e61104b57e7075a704d008
02/25/2021 18:51:58 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_id": 2,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id"

===== @#@!@##### ======
['/home/jovyan/Source_Code_Veri/data/GCJ/MS_C2T/gcj2017_java.jsonl']
===== @#@!@##### ======
/home/jovyan/Source_Code_Veri/data/GCJ/MS_C2T/gcj2017_java.jsonl


  0%|          | 0/2902 [00:00<?, ?it/s]

(256, 10, 768)


  0%|          | 1/2902 [00:24<19:54:48, 24.71s/it]

...

(256, 10, 768)


 13%|█▎        | 364/2902 [2:30:44<17:31:02, 24.85s/it]

(256, 10, 768)


 13%|█▎        | 364/2902 [2:30:59<17:32:46, 24.89s/it]


KeyboardInterrupt: 

In [29]:
save_all = model.predict[0] 
i = 0
if len(model.predict) >= 1:
    i += 1
    for batch in model.predict[1:]:
        save_all = np.concatenate((save_all, batch), axis=1)
    if i%10 == 0:
        print(i)
print(save_all.shape) 

(256, 10, 768)


In [32]:
save_all[:, 0, :]

array([[ 0.01590757, -0.18598336, -0.16474034, ...,  0.27126694,
        -0.3907697 ,  0.4303743 ],
       [-0.80239743,  0.04961689,  0.26808226, ..., -0.37215805,
        -0.32529736,  1.2891523 ],
       [-0.06264401,  0.30531552,  0.76245034, ..., -0.42283455,
        -0.37249666,  0.9501765 ],
       ...,
       [-0.2817436 ,  0.01992824,  0.2758034 , ..., -0.155825  ,
        -0.5058386 ,  0.5796027 ],
       [-0.03630484, -0.02493248,  0.2685474 , ...,  0.07700959,
        -0.44842076,  0.31877273],
       [ 0.0148202 , -0.18510294, -0.16449907, ...,  0.26991755,
        -0.3912418 ,  0.43141302]], dtype=float32)

In [6]:
import pandas as pd
import os

ds_all = pd.read_csv('/home/jovyan/Source_Code_Veri/data/GCJ/gcj2016.csv')
ds_all['language'] = ds_all.apply(lambda row: os.path.splitext(row['file'])[1][1:] , axis=1)
ds = ds_all.loc[ds_all.language=='py']

In [9]:
ds.iloc[0]['full_path']

'gcj/2016/4304486/xxifaye/5630113748090880/1/extracted/missing.py'

In [12]:
for i in range(ds.shape[0]):
    text = ds.iloc[i]['flines']
    try:
        text = text.replace('\n ', '\n')
    except:
        print('except')
        text = ''
    name = ds.iloc[i]['full_path'].replace('/', '_')
    with open('/home/jovyan/Source_Code_Veri/data/GCJ/C2V/files/2016/py/'+name, 'w') as f:
        f.write(text)

except
except
